In [1]:
#Import sqlite and tabulate for table formatting
import sqlite3
from tabulate import tabulate  

#Connect to DB
conn = sqlite3.connect('sql-murder-mystery.db')
cursor = conn.cursor()

#Set up clean tables and results with return messages (if needed)
def run_sql(query):
    """Run SQL and print results as a nice table"""
    try:
        cursor.execute(query)
        rows = cursor.fetchall()
        if rows:
            headers = [desc[0] for desc in cursor.description]
            print(tabulate(rows, headers=headers, tablefmt='psql'))
        else:
            print("Query executed. No rows returned.")
    except Exception as e:
        print(f"Error: {e}")

In [2]:
#Test query to return all tables
run_sql("""
SELECT name 
FROM sqlite_master 
WHERE type = 'table'
ORDER BY name;
""")

+------------------------+
| name                   |
|------------------------|
| crime_scene_report     |
| drivers_license        |
| facebook_event_checkin |
| get_fit_now_check_in   |
| get_fit_now_member     |
| income                 |
| interview              |
| person                 |
| solution               |
+------------------------+


In [3]:
#Viewing the crime scene report table to see format (first 5 rows)
run_sql("""
SELECT * 
FROM crime_scene_report 
LIMIT 5;
""")

+----------+---------+---------------------------------------------------------+----------+
|     date | type    | description                                             | city     |
|----------+---------+---------------------------------------------------------+----------|
| 20180115 | robbery | A Man Dressed as Spider-Man Is on a Robbery Spree       | NYC      |
| 20180115 | murder  | Life? Dont talk to me about life.                       | Albany   |
| 20180115 | murder  | Mama, I killed a man, put a gun against his head...     | Reno     |
| 20180215 | murder  | REDACTED REDACTED REDACTED                              | SQL City |
| 20180215 | murder  | Someone killed the guard! He took an arrow to the knee! | SQL City |
+----------+---------+---------------------------------------------------------+----------+


In [4]:
#Narrowing down for specific date and city mentioned on set up
run_sql("""
SELECT *
FROM crime_scene_report
WHERE date = '20180115'
AND city = 'SQL City';
""")

+----------+---------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+
|     date | type    | description                                                                                                                                                                               | city     |
|----------+---------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------|
| 20180115 | assault | Hamilton: Lee, do you yield? Burr: You shot him in the side! Yes he yields!                                                                                                               | SQL City |
| 20180115 | assault | Report Not Found                                                                         

In [5]:
#We have 2 witnesses, one who lives at the last house on NW Dr, and second named 'Annabel'
#who lives on Franklin Ave

In [6]:
#Viewing person table to see format
run_sql("""
SELECT *
FROM person
LIMIT 5;
""")

+-------+---------------------+--------------+------------------+-----------------------+-----------+
|    id | name                |   license_id |   address_number | address_street_name   |       ssn |
|-------+---------------------+--------------+------------------+-----------------------+-----------|
| 10000 | Christoper Peteuil  |       993845 |              624 | Bankhall Ave          | 747714076 |
| 10007 | Kourtney Calderwood |       861794 |             2791 | Gustavus Blvd         | 477972044 |
| 10010 | Muoi Cary           |       385336 |              741 | Northwestern Dr       | 828638512 |
| 10016 | Era Moselle         |       431897 |             1987 | Wood Glade St         | 614621061 |
| 10025 | Trena Hornby        |       550890 |              276 | Daws Hill Way         | 223877684 |
+-------+---------------------+--------------+------------------+-----------------------+-----------+


In [7]:
#Since we don't have full name for 'Annabel', searching for Annabel incomplete
run_sql("""
SELECT *
FROM person
WHERE name LIKE 'Annabel%' ;
""")

+-------+--------------------+--------------+------------------+-----------------------+-----------+
|    id | name               |   license_id |   address_number | address_street_name   |       ssn |
|-------+--------------------+--------------+------------------+-----------------------+-----------|
| 16371 | Annabel Miller     |       490173 |              103 | Franklin Ave          | 318771143 |
| 78354 | Annabell Siona     |       158932 |              978 | Whitewater Dr         | 800278294 |
| 78799 | Annabell Droneburg |       984316 |             1944 | W Natalie Dr          | 478793500 |
| 86541 | Annabell Zwilling  |       709133 |             1859 | Patti Rd              | 332961158 |
+-------+--------------------+--------------+------------------+-----------------------+-----------+


In [8]:
#We find Annabel Miller, the only one on Franklin Ave and with a single l
#which now gives us Annabel's full name and "keys"
run_sql("""
SELECT *
FROM person
WHERE name = 'Annabel Miller';
""")

+-------+----------------+--------------+------------------+-----------------------+-----------+
|    id | name           |   license_id |   address_number | address_street_name   |       ssn |
|-------+----------------+--------------+------------------+-----------------------+-----------|
| 16371 | Annabel Miller |       490173 |              103 | Franklin Ave          | 318771143 |
+-------+----------------+--------------+------------------+-----------------------+-----------+


In [9]:
#Now we find our second witness for more info using NW Dr
run_sql("""
SELECT *
FROM person
WHERE address_street_name = 'Northwestern Dr';
""")

+-------+----------------------+--------------+------------------+-----------------------+-----------+
|    id | name                 |   license_id |   address_number | address_street_name   |       ssn |
|-------+----------------------+--------------+------------------+-----------------------+-----------|
| 10010 | Muoi Cary            |       385336 |              741 | Northwestern Dr       | 828638512 |
| 12711 | Norman Apolito       |       667757 |              599 | Northwestern Dr       | 778264744 |
| 14887 | Morty Schapiro       |       118009 |             4919 | Northwestern Dr       | 111564949 |
| 15171 | Weldon Penso         |       336999 |              311 | Northwestern Dr       | 131379495 |
| 17729 | Lasonya Wildey       |       439686 |             3824 | Northwestern Dr       | 917817122 |
| 18376 | Josh Shi             |       653712 |             1091 | Northwestern Dr       | 193899001 |
| 19420 | Cody Schiel          |       890431 |             3524 | Northw

In [10]:
#A lot of results but, we know it is the last house so, we sort descending
run_sql("""
SELECT *
FROM person
WHERE address_street_name = 'Northwestern Dr'
ORDER BY address_number DESC;
""")

+-------+----------------------+--------------+------------------+-----------------------+-----------+
|    id | name                 |   license_id |   address_number | address_street_name   |       ssn |
|-------+----------------------+--------------+------------------+-----------------------+-----------|
| 14887 | Morty Schapiro       |       118009 |             4919 | Northwestern Dr       | 111564949 |
| 17729 | Lasonya Wildey       |       439686 |             3824 | Northwestern Dr       | 917817122 |
| 53890 | Sophie Tiberio       |       957671 |             3755 | Northwestern Dr       | 442830147 |
| 73368 | Torie Thalmann       |       773862 |             3697 | Northwestern Dr       | 341559436 |
| 96595 | Coretta Cubie        |       303645 |             3631 | Northwestern Dr       | 378403829 |
| 19420 | Cody Schiel          |       890431 |             3524 | Northwestern Dr       | 947110049 |
| 93509 | Emmitt Aceuedo       |       916706 |             3491 | Northw

In [11]:
#We find 'Morty Schapiro' as our second witness
run_sql("""
SELECT *
FROM person
WHERE id = '14887';
""")

+-------+----------------+--------------+------------------+-----------------------+-----------+
|    id | name           |   license_id |   address_number | address_street_name   |       ssn |
|-------+----------------+--------------+------------------+-----------------------+-----------|
| 14887 | Morty Schapiro |       118009 |             4919 | Northwestern Dr       | 111564949 |
+-------+----------------+--------------+------------------+-----------------------+-----------+


In [12]:
#Reviewing iterview table for format
run_sql("""
SELECT *
FROM interview
LIMIT 5;
""")

+-------------+-----------------------------------------------------------------------+
|   person_id | transcript                                                            |
|-------------+-----------------------------------------------------------------------|
|       28508 | ‘I deny it!’ said the March Hare.                                     |
|       63713 |                                                                       |
|       86208 | way, and the whole party swam to the shore.                           |
|       35267 | lessons in here? Why, there’s hardly room for YOU, and no room at all |
|       33856 |                                                                       |
+-------------+-----------------------------------------------------------------------+


In [13]:
#Reviewing Annabel's interview using person_id (fk)
run_sql("""
SELECT *
FROM interview
WHERE person_id = '16371';
""")

+-------------+-----------------------------------------------------------------------------------------------------------------------+
|   person_id | transcript                                                                                                            |
|-------------+-----------------------------------------------------------------------------------------------------------------------|
|       16371 | I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th. |
+-------------+-----------------------------------------------------------------------------------------------------------------------+


In [14]:
#Reviewing Morty's interview using person_id
run_sql("""
SELECT *
FROM interview
WHERE person_id = '14887';
""")

+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|   person_id | transcript                                                                                                                                                                                                                      |
|-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|       14887 | I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W". |
+-------------+-----------------

In [15]:
##We know the suspect has a "Get fit" gym membership number starting 48Z, is a gold member 
#and the vehicle plate starts with H42W and they were also at the gym on 01/09 and that
#the suspect is a man
##Since the gym is mentioned twice we can start there

In [16]:
#Reviewing the get_fit_now_check_in table
run_sql("""
SELECT *
FROM get_fit_now_check_in
LIMIT 5;
""")


+-----------------+-----------------+-----------------+------------------+
| membership_id   |   check_in_date |   check_in_time |   check_out_time |
|-----------------+-----------------+-----------------+------------------|
| NL318           |        20180212 |             329 |              365 |
| NL318           |        20170811 |             469 |              920 |
| NL318           |        20180429 |             506 |              554 |
| NL318           |        20180128 |             124 |              759 |
| NL318           |        20171027 |             418 |             1019 |
+-----------------+-----------------+-----------------+------------------+


In [17]:
#First we check the date to get a list of people there on that day w/ witness 1 (Annabel)
run_sql("""
SELECT *
FROM get_fit_now_check_in
WHERE check_in_date = 20180109;
""")

+-----------------+-----------------+-----------------+------------------+
| membership_id   |   check_in_date |   check_in_time |   check_out_time |
|-----------------+-----------------+-----------------+------------------|
| X0643           |        20180109 |             957 |             1164 |
| UK1F2           |        20180109 |             344 |              518 |
| XTE42           |        20180109 |             486 |             1124 |
| 1AE2H           |        20180109 |             461 |              944 |
| 6LSTG           |        20180109 |             399 |              515 |
| 7MWHJ           |        20180109 |             273 |              885 |
| GE5Q8           |        20180109 |             367 |              959 |
| 48Z7A           |        20180109 |            1600 |             1730 |
| 48Z55           |        20180109 |            1530 |             1700 |
| 90081           |        20180109 |            1600 |             1700 |
+-----------------+------

In [18]:
#We need to find witness 1's (Annabel) member number to match times
#Looking at the get_fit_now_member table for format
run_sql("""
SELECT * 
FROM get_fit_now_member
LIMIT 5;
""")

+-------+-------------+--------------------+-------------------------+---------------------+
| id    |   person_id | name               |   membership_start_date | membership_status   |
|-------+-------------+--------------------+-------------------------+---------------------|
| NL318 |       65076 | Everette Koepke    |                20170926 | gold                |
| AOE21 |       39426 | Noe Locascio       |                20171005 | regular             |
| 2PN28 |       63823 | Jeromy Heitschmidt |                20180215 | silver              |
| 0YJ24 |       80651 | Waneta Wellard     |                20171206 | gold                |
| 3A08L |       32858 | Mei Bianchin       |                20170401 | silver              |
+-------+-------------+--------------------+-------------------------+---------------------+


In [19]:
#Retrive a list of all people who were at the gym on that day and their membership info
run_sql("""
SELECT 
    c.check_in_date,
    c.membership_id,
    m.id AS member_id,
    m.person_id,
    m.name,
    m.membership_status
FROM get_fit_now_check_in c
INNER JOIN get_fit_now_member m 
    ON c.membership_id = m.id
WHERE c.check_in_date = '20180109'
ORDER BY m.name;
""")

+-----------------+-----------------+-------------+-------------+-------------------+---------------------+
|   check_in_date | membership_id   | member_id   |   person_id | name              | membership_status   |
|-----------------+-----------------+-------------+-------------+-------------------+---------------------|
|        20180109 | 1AE2H           | 1AE2H       |       10815 | Adriane Pelligra  | silver              |
|        20180109 | 90081           | 90081       |       16371 | Annabel Miller    | gold                |
|        20180109 | 7MWHJ           | 7MWHJ       |       31523 | Blossom Crescenzo | regular             |
|        20180109 | 6LSTG           | 6LSTG       |       83186 | Burton Grippe     | gold                |
|        20180109 | GE5Q8           | GE5Q8       |       92736 | Carmen Dimick     | gold                |
|        20180109 | 48Z55           | 48Z55       |       67318 | Jeremy Bowers     | gold                |
|        20180109 | 48Z7A   

In [20]:
#We found Annabel's info. Now we can use Annabel's person_id to determine 
#who was with her at the gym which was 1600-1700
run_sql("""
SELECT *
FROM get_fit_now_member
WHERE name = 'Annabel Miller';
""")

+-------+-------------+----------------+-------------------------+---------------------+
|    id |   person_id | name           |   membership_start_date | membership_status   |
|-------+-------------+----------------+-------------------------+---------------------|
| 90081 |       16371 | Annabel Miller |                20160208 | gold                |
+-------+-------------+----------------+-------------------------+---------------------+


In [21]:
# Using the previous table, we find these two people here, which match up our description
#from witness 2 (Morty) who said their gym membership started with '48Z'
#We now have two suspects names and person_id's!
run_sql("""
SELECT *
FROM get_fit_now_member
WHERE id = '48Z7A'
OR id = '48Z55';
""")

+-------+-------------+---------------+-------------------------+---------------------+
| id    |   person_id | name          |   membership_start_date | membership_status   |
|-------+-------------+---------------+-------------------------+---------------------|
| 48Z55 |       67318 | Jeremy Bowers |                20160101 | gold                |
| 48Z7A |       28819 | Joe Germuska  |                20160305 | gold                |
+-------+-------------+---------------+-------------------------+---------------------+


In [22]:
##We now have two suspects names who settle witness 1's (Annabel) account-
#-of seeing them on the gym for a specific day (01/09/2018).
##Drawing information from witness 2's (Morty) interview we matched the info (gym bag #'s) 
#and gold membership, which connect two suspects (Jeremy Bowers and Joe Germuska) both male
##Now we can take the final info from Morty's interview which is that the license plate
#started with 'H42W' and dig further

In [23]:
#Reviewing the drivers_license table for format
run_sql("""
SELECT *
FROM drivers_license
LIMIT 5;
""")

+--------+-------+----------+-------------+--------------+----------+----------------+------------+-------------+
|     id |   age |   height | eye_color   | hair_color   | gender   | plate_number   | car_make   | car_model   |
|--------+-------+----------+-------------+--------------+----------+----------------+------------+-------------|
| 100280 |    72 |       57 | brown       | red          | male     | P24L4U         | Acura      | MDX         |
| 100460 |    63 |       72 | brown       | brown        | female   | XF02T6         | Cadillac   | SRX         |
| 101029 |    62 |       74 | green       | green        | female   | VKY5KR         | Scion      | xB          |
| 101198 |    43 |       54 | amber       | brown        | female   | Y5NZ08         | Nissan     | Rogue       |
| 101255 |    18 |       79 | blue        | grey         | female   | 5162Z1         | Lexus      | GS          |
+--------+-------+----------+-------------+--------------+----------+----------------+--

In [24]:
#Searching for plate_number's that start with H42W
run_sql("""
SELECT *
FROM drivers_license
WHERE plate_number LIKE 'H42W%';
""")

+--------+-------+----------+-------------+--------------+----------+----------------+------------+-------------+
|     id |   age |   height | eye_color   | hair_color   | gender   | plate_number   | car_make   | car_model   |
|--------+-------+----------+-------------+--------------+----------+----------------+------------+-------------|
| 183779 |    21 |       65 | blue        | blonde       | female   | H42W0X         | Toyota     | Prius       |
+--------+-------+----------+-------------+--------------+----------+----------------+------------+-------------+


In [25]:
##Interesting, it gave us only one result but it is a female so, 
#there is possibly an accomplice! 

In [26]:
#Now that we have info on the vehicle and possible accomplice, we can search the
#license_id table and see who owns the vehicle
run_sql("""
SELECT *
FROM person
WHERE license_id = '183779';
""")

+-------+----------------+--------------+------------------+-----------------------+-----------+
|    id | name           |   license_id |   address_number | address_street_name   |       ssn |
|-------+----------------+--------------+------------------+-----------------------+-----------|
| 78193 | Maxine Whitely |       183779 |              110 | Fisk Rd               | 137882671 |
+-------+----------------+--------------+------------------+-----------------------+-----------+


In [27]:
#We see it is Maxine Whitely who lives on Fisk Rd. Since we don't have much more info
#to go on, we'll see if anyone has the same address
run_sql("""
SELECT *
FROM person
WHERE address_street_name = 'Fisk Rd';
""")

+-------+----------------+--------------+------------------+-----------------------+-----------+
|    id | name           |   license_id |   address_number | address_street_name   |       ssn |
|-------+----------------+--------------+------------------+-----------------------+-----------|
| 28819 | Joe Germuska   |       173289 |              111 | Fisk Rd               | 138909730 |
| 78193 | Maxine Whitely |       183779 |              110 | Fisk Rd               | 137882671 |
+-------+----------------+--------------+------------------+-----------------------+-----------+


In [28]:
#Very interesting, we see that Joe Germuska lives on Fisk Rd right next to
#Maxine. Possibly stole her vehicle to commit the crime?

In [29]:
#Lets check in on our other suspect and see where they live. 
run_sql("""
SELECT *
FROM person
WHERE name = 'Jeremy Bowers';
""")

+-------+---------------+--------------+------------------+-----------------------+-----------+
|    id | name          |   license_id |   address_number | address_street_name   |       ssn |
|-------+---------------+--------------+------------------+-----------------------+-----------|
| 67318 | Jeremy Bowers |       423327 |              530 | Washington Pl, Apt 3A | 871539279 |
+-------+---------------+--------------+------------------+-----------------------+-----------+


In [30]:
##Not a match for Jeremy Bowers on the address.
## Now we are out of 'leads' to go on and search through tables. 
##We have our two suspects (Jeremy Bowers and Joe Germuska) at the gym the same time 
#as our witness Annabel. 
##We have Maxine Whitely (possible 3rd suspect) who owns the vehicle that our second
#witness (Morty) viewed. And interesting enough Joe Germuska lives next door to that vehicle.
##Looking at the information (tables) available, we can se if they were interviewed
#and see what comes up for us to get more clues. 

In [31]:
#Seeing if Joe Germuska was interviewed
run_sql("""
SELECT * 
FROM interview
WHERE person_id = '28819';
""")

Query executed. No rows returned.


In [32]:
#Nothing for Joe. 
#Lets see if Maxine Whitely was interviewed
run_sql("""
SELECT * 
FROM interview
WHERE person_id = '78193';
""")

Query executed. No rows returned.


In [33]:
#Nothing for Maxine either
#Lets see for Jeremy Bowers
run_sql("""
SELECT * 
FROM interview
WHERE person_id = '67318';
""")

+-------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|   person_id | transcript                                                                                                                                                                                                                                       |
|-------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|       67318 | I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony 

In [34]:
##BOOM! We hit pay dirt and found 'our guy'! 
##But the plot thickens, he sang like a bird and stated there a mastermind to the 
#entire operation and gave us more information including gender (female), income (high),
#height range (65" to 67"), hair color (red), vehicle (Tesla Model S), and a date and time
#for location #(SQL Symphony Concert, 3 times in 12/2017). 
## Let's see plug this into our solutions table to confirm

In [35]:
run_sql("""
INSERT INTO solution (user, value)
VALUES (1, 'Jeremy Bowers');
""")

Query executed. No rows returned.


In [36]:
run_sql("""
SELECT value 
FROM solution;
""")

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| value                                                                                                                                                                                                                                                                                                                                                                                              |
|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [37]:
## We found and confirmed our murderer! Time to find our mastermind
#with added bonus if we can do it in 2 queries
##We know the driverse_license table has multiple pieces of information that
#our killer identified the mastermind with. 
##We have height, hair color, income, gender, car make and model. Lets see if we can 
#bring back something with that information


In [38]:
##Joining drivers_license and person tables with the pertinent info from our murderer
run_sql("""
SELECT 
    p.id AS person_id,
    p.name,
    p.address_number,
    p.address_street_name,
    p.ssn,
    d.id AS license_id,
    d.height,
    d.gender,
    d.hair_color,
    d.car_make,
    d.car_model,
    i.annual_income
FROM drivers_license d
INNER JOIN person p 
    ON d.id = p.license_id
LEFT JOIN income i 
    ON p.ssn = i.ssn
WHERE d.height IN (65, 66, 67)
  AND d.gender = 'female'
  AND d.hair_color = 'red'
  AND d.car_make = 'Tesla'
  AND d.car_model = 'Model S';
""")

+-------------+------------------+------------------+-----------------------+-----------+--------------+----------+----------+--------------+------------+-------------+-----------------+
|   person_id | name             |   address_number | address_street_name   |       ssn |   license_id |   height | gender   | hair_color   | car_make   | car_model   |   annual_income |
|-------------+------------------+------------------+-----------------------+-----------+--------------+----------+----------+--------------+------------+-------------+-----------------|
|       78881 | Red Korb         |              107 | Camerata Dr           | 961388910 |       918773 |       65 | female   | red          | Tesla      | Model S     |          278000 |
|       90700 | Regina George    |              332 | Maple Ave             | 337169072 |       291182 |       66 | female   | red          | Tesla      | Model S     |                 |
|       99716 | Miranda Priestly |             1883 | Golden Ave 

In [39]:
## We get 3 potential suspect results matching that description using demographics.
##Therefore we need to check into the symphony to view who visited 3 times
#in December of 2017.

In [40]:
##Here we count the checkins from the facebook_event_checkin table to determine who has 3
#'check ins' during the month/ year combo of 12/2017 for the event
#to include SQL Symphony. 
#Then we are also checking those results against our 3 suspects id's to see if we get a hit
run_sql("""
SELECT person_id, COUNT(*) AS checkin_count
FROM facebook_event_checkin
WHERE event_name LIKE 'SQL Symphony%'
  AND date LIKE '201712%'
  AND person_id IN (99716, 90700, 78881)
GROUP BY person_id
HAVING COUNT(*) = 3;
""")

+-------------+-----------------+
|   person_id |   checkin_count |
|-------------+-----------------|
|       99716 |               3 |
+-------------+-----------------+


In [41]:
##And there we have it! Only one person of the 3 suspects visited the SQL Symphony 
#in December of 2017 and it matches to our previous table showing 'Miranda Priestly'
##That is our suspected mastermind and we can check to confirm. 

In [42]:
run_sql("""
INSERT INTO solution (user, value)
VALUES (1, 'Miranda Priestly');
""")

Query executed. No rows returned.


In [43]:
run_sql("""
SELECT value 
FROM solution;
""")

+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
| value                                                                                                                                                        |
|--------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne! |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------+


In [44]:
conn.close()